## Test with straight beam

The aim of this test is to compare the inertial loads of an analytical case with the ones given by the code.

The beam used is straight, measures 1 meter, weighs 100 kg and it only has two nodes. It is defined with the following half chord position and property files.


The stiffness matrix is not needed in this test, but, to make the structural code work, a valid matrix was used from the first node of the original IEA 15 MW reference WT file.

In [28]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from beam_corot.ComplBeam import ComplBeam
from beam_corot.CoRot import CoRot
from utils import save_load
from inertial_forces_v2 import inertial_loads_fun, inertial_loads_fun_v02, inertial_loads_v03


In [29]:

# Model input json file name
f_model_json = "straight_beam_no_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# Extra geometry info
hub_di = 0 # Hub diameter [m]
# distance of the beam from the axis of rotation

In [30]:
# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_no_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_no_offset.nodeLocations[:,2] # z-axis position
beam_no_offset.nodeLocations

-----------
ComplBeam Model Created
Static analysis done


array([[0., 0., 0.],
       [0., 0., 1.]])

In [31]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_no_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_no_offset = ComplBeam(mainfile_dyn)
M_mat_no_offset = beam_no_offset.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_no_offset_df = pd.DataFrame(M_mat_no_offset)
M_mat_no_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.000000,0.042634,4.209301,-0.093673,16.499820,0.207198,0.000000,0.042634,-4.124033,0.093673
1,-0.207198,33.580883,0.000000,-4.229476,-0.062810,-0.137028,0.207198,16.419117,0.000000,4.103857,-0.062810,0.137028
2,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000
3,0.042634,-4.229476,0.000000,0.834256,0.000922,0.000751,-0.042634,-4.103857,0.000000,-0.832411,0.000922,-0.000751
4,4.209301,-0.062810,0.000000,0.000922,0.834256,0.000751,4.124033,0.062810,0.000000,0.000922,-0.832411,-0.000751
5,-0.093673,-0.137028,0.000000,0.000751,0.000751,0.004702,0.093673,0.137028,0.000000,0.000751,0.000751,-0.004702
6,16.499820,0.207198,0.000000,-0.042634,4.124033,0.093673,33.500180,-0.207198,0.000000,-0.042634,-4.209301,-0.093673
7,0.207198,16.419117,0.000000,-4.103857,0.062810,0.137028,-0.207198,33.580883,0.000000,4.229476,0.062810,-0.137028
8,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
9,0.042634,4.103857,0.000000,-0.832411,0.000922,0.000751,-0.042634,4.229476,0.000000,0.834256,0.000922,-0.000751


## Calculating centrifugal force

Let's calculate the acceleration in the node position which is also the CoG.

The operational condicions are:

    RPM = 7
    pitch = 0


In [32]:
# Operations info
omega = 1 # rad/s
pitch_deg = 0
pitch_rad = np.deg2rad(pitch_deg)


First, the posotions are needed. These are set manually because it is very simple.

In [33]:
pos = np.array([[0,0,0,0,0,0],[0,0,1,0,0,0]])
pos = pos.flatten()

In [34]:
# Calculate inertial loads
load = -inertial_loads_fun(pos,M_mat_no_offset,hub_di,omega,pitch_rad)
load = np.reshape(load, (-1,6))
load

array([[-0.        , -0.        , 16.66666667, -0.        , -0.        ,
        -0.        ],
       [-0.        , -0.        , 33.33333333, -0.        , -0.        ,
        -0.        ]])

## Analytical result

This problem is really simple on paper. There is a beam measuring 1 meter, weighing a 100 kg and rotating at a constant 1 rad/s around one of its ends.

The centrifugar force in its center of mass is $F=m\cdot \omega^{2}\cdot r= 100 \cdot 1 \cdot 0.5 = 50 \text{ N}$. This force is outwards, meaning positive z.

This is equal to the summe of the force of both nodes!

## Adding an offset in the centre of gravity

The beam has an offset of the centre of gravity of 10 cm in the x direction.

The mass matrix can be obtained the same way.

In [35]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_2m = ComplBeam(mainfile_dyn)
M_mat_2m = beam_2m.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_2m_df = pd.DataFrame(M_mat_2m)
M_mat_2m_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.244735,0.042634,4.209301,-0.105866,16.499820,0.207198,0.244735,0.042634,-4.124033,0.105866
1,-0.207198,33.580883,-0.244735,-4.229476,-0.062810,3.208498,0.207198,16.419117,-0.244735,4.103857,-0.062810,1.791502
2,0.244735,-0.244735,33.333333,0.122368,-3.210966,-0.285525,-0.244735,0.244735,16.666667,0.122368,-1.544299,0.285525
3,0.042634,-4.229476,0.122368,0.834256,0.000922,-0.422012,-0.042634,-4.103857,0.122368,-0.832411,0.000922,-0.411321
4,4.209301,-0.062810,-3.210966,0.000922,0.834256,-0.005346,4.124033,0.062810,-1.544299,0.000922,-0.832411,0.005346
5,-0.105866,3.208498,-0.285525,-0.422012,-0.005346,-0.022404,0.105866,1.791502,-0.285525,0.411321,-0.005346,0.022404
6,16.499820,0.207198,-0.244735,-0.042634,4.124033,0.105866,33.500180,-0.207198,-0.244735,-0.042634,-4.209301,-0.105866
7,0.207198,16.419117,0.244735,-4.103857,0.062810,1.791502,-0.207198,33.580883,0.244735,4.229476,0.062810,3.208498
8,0.244735,-0.244735,16.666667,0.122368,-1.544299,-0.285525,-0.244735,0.244735,33.333333,0.122368,-3.210966,0.285525
9,0.042634,4.103857,0.122368,-0.832411,0.000922,0.411321,-0.042634,4.229476,0.122368,0.834256,0.000922,0.422012


Here we are going to compare two ways of using this mass matrix. We can multiply it by the accelerations in the node positions or in the centre of mass of the nodes.

In [36]:

# Calculate inertial loads using the node accelerations (positions in our input)
load_4 = -inertial_loads_fun(pos,M_mat_2m,hub_di,omega,pitch_rad)
load_4 = np.reshape(load_4, (-1,6))
load_4

array([[ 0.24473539, -0.24473539, 16.66666667,  0.12236769, -1.54429897,
        -0.28552462],
       [-0.24473539,  0.24473539, 33.33333333,  0.12236769, -3.21096564,
         0.28552462]])

In [37]:
# Calculate inertial loads using the CoG accelerations (positions in our input)
pos_cg = np.array([[0.1,0,0,0,0,0],[0.1,0,1,0,0,0]])
print(f'{pos_cg = }')

load_2 = -inertial_loads_fun(pos_cg,M_mat_2m,hub_di,omega,pitch_rad)
load_2 = np.reshape(load_2, (-1,6))
load_2

pos_cg = array([[0.1, 0. , 0. , 0. , 0. , 0. ],
       [0.1, 0. , 1. , 0. , 0. , 0. ]])


array([[ 5.24473539, -0.24473539, 16.66666667,  0.12236769, -0.71096564,
        -0.28552462],
       [ 4.75526461,  0.24473539, 33.33333333,  0.12236769, -4.04429897,
         0.28552462]])

## Analytical result

The offset just moves the centre of gravity to the position $(0.1, 0, 0.5)$. This leads to the inertial force $F = (10, 0, 50)$ N being applied to this point. While the moment is zero.

Let's now calculate the equivalent load in that point from the two nodes.

In [38]:
force = load_4[:,:3]
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')
moment = load_4[:,3:]
r_cg2node = np.array([[-0.1,  0. , -0.5],[-0.1,  0. , 0.5]])
moment_cg = np.sum(moment,axis=0) + np.cross(r_cg2node[0],force[0]) + np.cross(r_cg2node[1],force[1])
print(f'{moment_cg = }')

force_cg = array([ 0.,  0., 50.])
moment_cg = array([ 2.93903790e-13, -3.06421555e-13,  0.00000000e+00])


As you can see, eventhough there is an offset, the x-axis load is zero. Regarding the moments in the CoG, they are low enough to be consider zero.

Now let's calculate the inertial loads using the acceleration in the CoG.

In [39]:
force = load_2[:,:3]
# Total force: summe of the force vectors
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')
moment = load_2[:,3:]
r_cg2node = np.array([[-0.1,  0. , -0.5],[-0.1,  0. , 0.5]])
# Total moment: summe of the moment vectors and the moment arm of the node forces, (r_cg2F x F)
moment_cg = np.sum(moment,axis=0) + np.cross(r_cg2node[0],force[0]) + np.cross(r_cg2node[1],force[1])
print(f'{moment_cg = }')

force_cg = array([1.00000000e+01, 2.77555756e-17, 5.00000000e+01])
moment_cg = array([-5.20278265e-14, -8.53539461e-13, -3.46944695e-18])


Now the resulting force in the CoG has the x-axis compenet we are expecting. Furthermore, the moments continue being small enough to be consider zero.

## Integration

Next step we can built another `inertial_loads_fun` that integrates the offset and the translation of the loads to the nodes.

In [42]:
# Inertial loads in the centre of gravity.
# Asuming the accelerations are in the CoG and the forces calculated are there too

pos = np.array([[0.,0.,0.,0.,0.,0.],[0.,0.,1.,0.,0.,0.]])

offset = np.array([[0.1,0,0],[0.1,0,0]])

pos_cg = pos.copy()
pos_cg[:,:3] += offset

print(f'{pos = }')
print(f'{pos_cg = }')


pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])
pos_cg = array([[0.1, 0. , 0. , 0. , 0. , 0. ],
       [0.1, 0. , 1. , 0. , 0. , 0. ]])


In [45]:
pos2acc = np.array([-omega**2, 0, -omega**2, 0, 0, 0])

acc_mat = pos_cg * pos2acc

acc_vec = acc_mat.flatten()

inertial_loads_cg = M_mat_2m @ acc_vec

inertial_loads_cg = np.reshape(inertial_loads_cg, (-1,6))

-inertial_loads_cg

force = inertial_loads_cg[:,:3]
moment = inertial_loads_cg[:,3:]
# force, moment = np.split(inertial_loads_cg,[3],axis=1)
# Translating the moments to the node positions
# for node_i in range(pos.shape[0]):
#     moment += np.cross(offset[node_i,:],force[node_i,:])

inertial_loads = np.concatenate((force,moment),axis=1)

print(f'{force = }')
print(f'{moment = }')
print(f'{inertial_loads = }')


force = array([[ -5.24473539,   0.24473539, -16.66666667],
       [ -4.75526461,  -0.24473539, -33.33333333]])
moment = array([[-0.24473539,  6.42193128,  0.57104924],
       [-0.24473539, 13.08859795, -0.57104924]])
inertial_loads = array([[ -5.24473539,   0.24473539, -16.66666667,  -0.24473539,
          6.42193128,   0.57104924],
       [ -4.75526461,  -0.24473539, -33.33333333,  -0.24473539,
         13.08859795,  -0.57104924]])


Finally, this algorithm is integrated in the function. Here it is tested that both results are the same for this example.

In [46]:
pos = np.array([[0.,0.,0.,0.,0.,0.],[0.,0.,1.,0.,0.,0.]])
offset = np.array([[0.1,0,0],[0.1,0,0]])

inertial_loads_fun = inertial_loads_fun_v02(pos,offset,M_mat_2m,omega)

inertial_loads_fun = np.reshape(inertial_loads_fun, (-1,6))

if np.array_equal(inertial_loads, inertial_loads_fun): print('The two results are the same.')
else: print('ERROR: The two results are the different.')

ERROR: The two results are the different.


## Adding the pitch

First we check that the pitch implementation gives the same result in the previous examples, which is the case.

In [ ]:
res_inertial_loads_v03 = inertial_loads_v03(pos,offset,M_mat_2m,0,omega,0)

res_inertial_loads_v03 = np.reshape(res_inertial_loads_v03, (-1,6))

if np.array_equal(inertial_loads, res_inertial_loads_v03): print('The two results are the same.')
else: print('ERROR: The two results are the different.')

ERROR: The two results are the different.


Now lets create a new example. In this case the blade has an offset of 10 cm in the y-axis, but a pitch of 90 degrees. Furthermore, the root is place at 1 meter from the rotating axis of the rotor.

The position of the CoG now is $(0.1,0,1.5)$ in the rotor frame of reference. The force then is $F=(10,0,150)$ in rotor frame of reference and the moment is zero. However, the output is in the blade frame of reference, thus $F=(0,10,150)$

In [ ]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_y_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_2m = ComplBeam(mainfile_dyn)
M_mat_2m = beam_2m.M_mat_full
pos = np.concatenate((beam_2m.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

# Display DataFrame in Jupyter notebook
M_mat_2m_df = pd.DataFrame(M_mat_2m)
M_mat_2m_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,-0.365790,0.042634,4.209301,-3.435164,16.499820,0.207198,-0.365790,0.042634,-4.124033,-1.564836
1,-0.207198,33.580883,0.365790,-4.229476,-0.062810,-0.128870,0.207198,16.419117,0.365790,4.103857,-0.062810,0.128870
2,-0.365790,0.365790,33.333333,3.150439,-0.182895,-0.406579,0.365790,-0.365790,16.666667,1.483772,-0.182895,0.406579
3,0.042634,-4.229476,3.150439,0.834256,0.000922,-0.003328,-0.042634,-4.103857,1.483772,-0.832411,0.000922,0.003328
4,4.209301,-0.062810,-0.182895,0.000922,0.834256,-0.419995,4.124033,0.062810,-0.182895,0.000922,-0.832411,-0.413339
5,-3.435164,-0.128870,-0.406579,-0.003328,-0.419995,0.023737,-1.564836,0.128870,-0.406579,-0.003328,0.413339,-0.023737
6,16.499820,0.207198,0.365790,-0.042634,4.124033,-1.564836,33.500180,-0.207198,0.365790,-0.042634,-4.209301,-3.435164
7,0.207198,16.419117,-0.365790,-4.103857,0.062810,0.128870,-0.207198,33.580883,-0.365790,4.229476,0.062810,-0.128870
8,-0.365790,0.365790,16.666667,1.483772,-0.182895,-0.406579,0.365790,-0.365790,33.333333,3.150439,-0.182895,0.406579
9,0.042634,4.103857,1.483772,-0.832411,0.000922,-0.003328,-0.042634,4.229476,3.150439,0.834256,0.000922,0.003328


In [ ]:
offset = np.array([[0,0.1,0],[0,0.1,0]])

# Operations info
omega = 1 # rad/s
pitch_deg = 90
pitch_rad = np.deg2rad(pitch_deg)

r_hub = 1

In [ ]:
load_4 = -inertial_loads_v03(pos,offset,M_mat_2m,r_hub*2,omega,pitch_rad)

load_4 = np.reshape(load_4, (-1,6))

print(f'{load_4 = }')

load_4 = array([[-1.09736876,  6.09736876, 66.66666667, 11.95131562, -0.54868438,
        -1.10999957],
       [ 1.09736876,  3.90263124, 83.33333333, 16.95131562, -0.54868438,
         1.10999957]])


In [ ]:
force, moment = np.split(load_4,[3],axis=1)
# Total force: summe of the force vectors
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')
r_cg2node = np.array([[0,  -0.1 , -0.5],[0.,  -0.1 , 0.5]])
# Total moment: summe of the moment vectors and the moment arm of the node forces, (r_cg2F x F)
moment_cg = np.sum(moment,axis=0) + np.cross(r_cg2node[0],force[0]) + np.cross(r_cg2node[1],force[1])
print(f'{moment_cg = }')

force_cg = array([-6.66133815e-16,  1.00000000e+01,  1.50000000e+02])
moment_cg = array([ 1.50000000e+01, -1.16018306e-13,  1.52655666e-16])


In [ ]:
force, moment = np.split(load_4,[3],axis=1)
# Total force: summe of the force vectors
force_cg = np.zeros(3)
for i in range(3):
    force_cg[i] = np.trapz(force[:,i],pos[:,2])
print(f'{force_cg = }')
length = pos[-1,2]-pos[0,2]
print(f'{length = }')
moment_cg = length**2/12*(force[0,:]-force[1,:])
for i in range(3):
    moment_cg[i] += np.trapz(moment[:,i],pos[:,i])
print(f'{moment_cg = }')

force_cg = array([-3.33066907e-16,  5.00000000e+00,  7.50000000e+01])
length = 1.0
moment_cg = array([-0.18289479,  0.18289479, -1.38888889])


In [ ]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_2m_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_2m = ComplBeam(mainfile_dyn)
M_mat_2m = beam_2m.M_mat_full
pos = np.concatenate((beam_2m.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

# Display DataFrame in Jupyter notebook
M_mat_2m_df = pd.DataFrame(M_mat_2m)
M_mat_2m_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0.]])


,0,1,2,3,4,5,6,7,8,9,10,11
0,67.685220,-1.254818,0.000000,0.540898,17.207565,-0.765513,32.314780,1.254818,0.000000,0.540898,-16.125768,0.765513
1,-1.254818,68.157749,0.000000,-17.443829,-0.777163,-1.062936,1.254818,31.842251,0.000000,15.889504,-0.777163,1.062936
2,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
3,0.540898,-17.443829,0.000000,6.729910,0.063243,0.030580,-0.540898,-15.889504,0.000000,-6.603424,0.063243,-0.030580
4,17.207565,-0.777163,0.000000,0.063243,6.729910,0.030580,16.125768,0.777163,0.000000,0.063243,-6.603424,-0.030580
5,-0.765513,-1.062936,0.000000,0.030580,0.030580,0.145673,0.765513,1.062936,0.000000,0.030580,0.030580,-0.145673
6,32.314780,1.254818,0.000000,-0.540898,16.125768,0.765513,67.685220,-1.254818,0.000000,-0.540898,-17.207565,-0.765513
7,1.254818,31.842251,0.000000,-15.889504,0.777163,1.062936,-1.254818,68.157749,0.000000,17.443829,0.777163,-1.062936
8,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000
9,0.540898,15.889504,0.000000,-6.603424,0.063243,0.030580,-0.540898,17.443829,0.000000,6.729910,0.063243,-0.030580


In [ ]:

# Calculate inertial loads using the node accelerations (positions in our input)

offset = np.zeros((2,3))
load_5 = -inertial_loads_v03(pos,offset,M_mat_2m,0,omega,0)
load_5 = np.reshape(load_4, (-1,6))
print(f'{load_4 = }')

load_4 = array([[-1.09736876,  6.09736876, 66.66666667, 11.95131562, -0.54868438,
        -1.10999957],
       [ 1.09736876,  3.90263124, 83.33333333, 16.95131562, -0.54868438,
         1.10999957]])


In [ ]:

force, moment = np.split(load_5,[3],axis=1)
# Total force: summe of the force vectors
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')
r_cg2node = np.array([[0,  0 , -0],[0.,  -0. , 0.]])
# Total moment: summe of the moment vectors and the moment arm of the node forces, (r_cg2F x F)
moment_cg = np.sum(moment,axis=0) + np.cross(r_cg2node[0],force[0]) + np.cross(r_cg2node[1],force[1])
print(f'{moment_cg = }')

force_cg = array([-6.66133815e-16,  1.00000000e+01,  1.50000000e+02])
moment_cg = array([ 2.89026312e+01, -1.09736876e+00,  2.22044605e-16])


This confirms that the loads are given in N/m and not N. The bar is 100 kg/m, thus it weighs 200 kg. Therefore the force should be 200 N.

In [ ]:
force, moment = np.split(load_5,[3],axis=1)
# Total force: summe of the distributed force along z-axis
force_cg = np.zeros(3)
for i in range(3):
    force_cg[i] = np.trapz(force[:,i],pos[:,2])
print(f'{force_cg = }')
length = pos[-1,2]-pos[0,2]
print(f'{length = }')
moment_cg = length**2/12*(force[0,:]-force[1,:])
for i in range(3):
    moment_cg[i] += np.trapz(moment[:,i],pos[:,2])
print(f'{moment_cg = }')

force_cg = array([-6.66133815e-16,  1.00000000e+01,  1.50000000e+02])
length = 2.0
moment_cg = array([28.17105207, -0.36578959, -5.55555556])
